In [1]:
import pandas as pd
import numpy as np
import math as m
import random as rand
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime
from sklearn import linear_model as lm, metrics, ensemble as ens
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.svm import SVC
from sklearn.feature_selection import RFE, RFECV, SequentialFeatureSelector
import random
from scipy import stats

# from collections import defaultdict
# from itertools import islice
# from scipy.stats import permutation_test
# import statsmodels.api as sm
# from statsmodels.genmod.generalized_linear_model import GLM

In [2]:
#DEFINING A FUNCTION TO UPDATE COLUMN NAMES LATER
def lower_no_space(word): 
    
    word = re.sub(' ', '_', word) 
    
    word = re.sub(r'\'', '', word) 
    
    word = re.sub(r'\(', '', word)
    
    word = re.sub(r'\)', '', word)
    
    word = re.sub('\?', '', word)
    
    word = re.sub('/', '_', word)
    
    word = word.lower()
    
    return word

In [3]:
df_gen = pd.read_csv("full_gen.csv")

In [4]:
#READ IN Updated CLINICAL DATA FOR LATER USE (CONVERTED TO .csv IN GOOGLE SHEETS)
df_clin_updated = pd.read_csv("Homebase_new_updated.csv", header = 1)

In [5]:
#RENAMING COLUMNS
df_clin_updated = df_clin_updated.rename(mapper = lower_no_space, axis = 1) 
df_clin_updated.rename(columns={'subject_sample_id':'sample_id'}, inplace=True)

In [6]:
#Compute the age at initial diagnosis from date of birth and date_of_initial_diagnosis
df_clin_updated['date_of_birth'] = pd.to_datetime(df_clin_updated['date_of_birth'])
df_clin_updated['date_of_initial_diagnosis'] = pd.to_datetime(df_clin_updated['date_of_initial_diagnosis'])
df_clin_updated["age_at_initial_diagnosis"] = (pd.DatetimeIndex(df_clin_updated['date_of_initial_diagnosis']).year 
                        - pd.DatetimeIndex(df_clin_updated['date_of_birth']).year)

In [7]:
#Due to the abnormal in date of birth from the Stanford data, 
#Remove the age at initial diagonosis for data from Stanford & the one that has negative age 
df_clin_updated["age_at_initial_diagnosis"] = np.where(df_clin_updated['data_access_group'] == 'Stanford', np.nan, df_clin_updated["age_at_initial_diagnosis"])
df_clin_updated["age_at_initial_diagnosis"] = np.where(df_clin_updated["age_at_initial_diagnosis"] < 0, np.nan, df_clin_updated["age_at_initial_diagnosis"])


In [8]:
#Change the data type: date_of_birth, n, m 
df_clin_updated = df_clin_updated.astype({'t':'object', 'b':'object'})


In [9]:
#TONS OF DATA, PULL WHAT WE WANT
df_clin_updated_lean = df_clin_updated.drop(columns = [x for x in df_clin_updated.columns if x not in ['gender', 'race', \
                                       'country_of_residence', 'sample_id', 'ethnicity',\
                                        'age_at_initial_diagnosis', 't', 'n', 'm', 'b',\
                                        'predominant_lesion_type_at_diagnosis','lymph_node_biopsy_performed',\
                                        'family_history_of_leukemia_lymphoma', \
                                        'has_the_patient_ever_been_exposed_at_work_or_in_the_service_to_a_toxic_chemical',\
                                        'cd4+:cd8+_ratio', 'total_lymphocyte_count', 'absolute_cd4+_count_per_ul',\
                                        '%cd4+cd26-', '%cd4+cd7-', 'tcr_clonality', 'tumor_cell_cd30+',\
                                        'large_cell_transformation', 'ldh_u_l', 'wbc_103_μl', 'rbc_106_μl',\
                                        'hematocrit_%', 'mcv_fl', 'mchc_g_dl', 'rdw_%', 'platelet_count_103_μl',\
                                        'segmented_neutrophil,_absolute_103_μl', 'lymphocyte,_absolute_103_μl',\
                                        'monocytes,_absolute_103_μl', 'eosinophils,_absolute_103_μl',\
                                        'basophils,_absolute_103_μl', 'segmented_neutrophils_%', 'lymphocytes_%',\
                                        'monocytes_%', 'eosinophils_%', 'basophils_%']])

In [10]:
# TURN YES/NO & POSITIVE/NEGATIVE TO DUMMIES
df_clin_updated_lean['lymph_node_biopsy_performed'] = \
df_clin_updated_lean['lymph_node_biopsy_performed'].apply(lambda x: 1 if x == 'Yes' else 0)

df_clin_updated_lean['family_history_of_leukemia_lymphoma'] = \
df_clin_updated_lean['family_history_of_leukemia_lymphoma'].apply(lambda x: 1 if x == 'Yes' else 0)

df_clin_updated_lean['tumor_cell_cd30+'] = \
df_clin_updated_lean['tumor_cell_cd30+'].apply(lambda x: 1 if x == 'Yes' else 0)

df_clin_updated_lean['large_cell_transformation'] = \
df_clin_updated_lean['large_cell_transformation'].apply(lambda x: 1 if x == 'Yes' else 0)

df_clin_updated_lean['tcr_clonality'] = \
df_clin_updated_lean['tcr_clonality'].apply(lambda x: 1 if x == 'Positive' else 0)

df_clin_updated_lean['has_the_patient_ever_been_exposed_at_work_or_in_the_service_to_a_toxic_chemical'] = \
df_clin_updated_lean['has_the_patient_ever_been_exposed_at_work_or_in_the_service_to_a_toxic_chemical'].apply(lambda x: 1 if x == 'Yes' else 0)

/var/folders/pm/58gwkyt94zlcnzlf70kv3yt80000gn/T/ipykernel_68605/371693475.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clin_updated_lean['lymph_node_biopsy_performed'] = \
/var/folders/pm/58gwkyt94zlcnzlf70kv3yt80000gn/T/ipykernel_68605/371693475.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clin_updated_lean['family_history_of_leukemia_lymphoma'] = \
/var/folders/pm/58gwkyt94zlcnzlf70kv3yt80000gn/T/ipykernel_68605/371693475.py:8: SettingWithCopyWarning: 
A value is trying to be set on a

In [34]:
# Read in the Preprocessed Genetic Data
df_lean = pd.read_csv ('stats_by_sample.csv')

In [35]:
df_lean.head()

,Unnamed: 0,sample_id,gene_1-Mar_rawscore,gene_1-Sep_rawscore,gene_11-Mar_rawscore,gene_2-Mar_rawscore,gene_2-Sep_rawscore,gene_4-Mar_rawscore,gene_4-Sep_rawscore,gene_5-Sep_rawscore,...,chromosome_1_mutations,chromosome_8_mutations,chromosome_Y_mutations,chromosome_5_mutations,chromosome_3_mutations,chromosome_11_mutations,chromosome_2_mutations,chromosome_6_mutations,chromosome_22_mutations,outcome
0,0,11__ungewickell__SS,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0,3,0,0,0,4,2,2,0,1
1,1,15__ungewickell__SS,0.0,0.0,0.0,0.0,2.511819,0.0,0.0,0.0,...,2,1,0,1,1,5,4,1,1,1
2,2,16__ungewickell__MF,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,8,0,0,1,3,3,3,3,0,0
3,3,1__ungewickell__SS,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,1,0,0,1,4,2,1,0,1,1
4,4,4__ungewickell__SS,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,6,4,0,5,3,2,0,1,0,1


In [12]:
#TRANSFORM SAMPLE ID TO JOIN TO CLINICAL DATA
df_lean['sample_id'] = df_lean['sample_id'].apply(lambda x: re.sub('_', '-', x[:5]) if 'WES' in x else\
                                                  (x[:-10] if 'CTCL' in x else \
                                                  (x[:-13] if 'almeida' in x else\
                                                  ((x[-2:]+x[:-2])[:-15] if 'ungewickell' in x else\
                                                  ('-'.join([ele.lstrip('0').lower() for ele in x[:-10].split('-')]) if 'SPZ' in x else x)))))

In [13]:
#MERGE tbe updated CLINICAL, GENETIC DATA
df_all_updated = pd.merge(df_lean, df_clin_updated_lean, on='sample_id', how='left')

In [14]:
#IMPUTATION; "UNKNOWN" FOR CATEGORICAL, MEAN FILL-IN FOR CONTINUOUS
for col in df_clin_updated_lean.columns:
    if col in ['race', 'gender', 'country_of_residence', 'ethnicity', 'predominant_lesion_type_at_diagnosis', 't', 
              'n', 'm', 'b']:
        df_all_updated[col] = df_all_updated[col].fillna('unknown')
    elif col != 'sample_id':
        df_all_updated[col] = df_all_updated[col].fillna(np.mean(df_all_updated[col]))

In [15]:
#GET DUMMIES FOR CATEGORICALS
df_all_updated = pd.get_dummies(df_all_updated, columns = ['race', 'gender', 'country_of_residence', 'ethnicity', 'predominant_lesion_type_at_diagnosis', 
                                                          't', 'n', 'm', 'b'])


In [16]:
#DEFINE STANDARDSCALER FOR LATER USE
std_scl = StandardScaler()

In [17]:
# Define (Scaled/Normalized) Features and Labels
X_new = df_all_updated.drop(columns = [x for x in df_all_updated.columns if x == 'outcome' or x == 'sample_id'])
X_new_scaled = std_scl.fit_transform(X_new)
X_new_norm = normalize(X_new)

y_new = df_all_updated.drop(columns = [x for x in df_all_updated.columns if x != 'outcome'])

In [18]:
df_all_updated['outcome'].value_counts(normalize = True)

1    0.8
0    0.2
Name: outcome, dtype: float64

In [19]:
# DEFINE CHROMOSOMES OF INTEREST
top_feats = pd.read_csv("ada_fi.csv").head(25).drop(columns = ['Unnamed: 0'])
top_feats
chroms_to_use = top_feats[top_feats['feature_names'].str.contains("chromosome")]['feature_names']
chroms_to_use = [re.sub('chromosome_', '', \
                        re.sub('_mutations', '', \
                               re.sub('_rawscore', '', \
                                      re.sub('_non_neg_rawscore', '', \
                                             re.sub('med_', '', x))))) \
                 for x in chroms_to_use]


chroms_to_use_2 = top_feats[(top_feats['feature_names'].str.contains("section")) &\
                           ~(top_feats['feature_names'].str.contains("chromosome"))]['feature_names']
chroms_to_use_2 = [re.sub('_mutations', '', \
                          re.sub('_rawscore', '', \
                                 re.sub('_non_neg_rawscore', '', \
                                        x.split("chrom_", 1)[1])))\
                  for x in chroms_to_use_2]


chroms_to_use += chroms_to_use_2
chroms_to_use = list(set(chroms_to_use))
chroms_to_use

['18',
 '12',
 '4',
 '5',
 '17',
 '16',
 '11',
 '3',
 '6',
 '7',
 '1',
 '15',
 '13',
 '19',
 'X']

In [20]:
df_use = df_all_updated.copy()
cols_to_use = ['outcome']

for gene in set(df_gen[df_gen['chrom'].isin(chroms_to_use)]['gene_symbol']):
    if ('gene_' + gene + '_non_neg_rawscore') in df_lean.columns:
            cols_to_use.append('gene_' + gene + '_non_neg_rawscore')
df_use = df_use.drop(columns = [x for x in df_use.columns if x not in cols_to_use])

In [21]:
log = lm.LogisticRegression()
rf = ens.RandomForestClassifier()
rdg = lm.RidgeClassifier(alpha = 1)
ada = ens.AdaBoostClassifier()


NEXT FEW CELLS ARE ABOUT GETTING P-VALS FROM SKLEARN;
METHODOLOGY LEARNED HERE:
https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression

In [22]:
X = df_use.drop(columns = [x for x in df_use.columns if x == 'sample_id' or x == 'outcome'])
y = df_use['outcome']

In [23]:
class perm_test():
    def linear(self, model_type, ex, why):
        
        model = model_type.fit(ex, why.values.ravel())
        
        # DEFINE FULL MODEL PARAMS (INCLUDE B_0) AND PREDICTIONS YOU WOULD GET
        params = np.append(model.intercept_, model.coef_)
        predictions = model.predict(ex)
        
        # MAKE NEW DF TO STORE KEY VALUES, DEFINE MSE OF PREDS FROM PREV CELL
        newX = pd.DataFrame({"Constant":np.ones(len(ex))}).join(pd.DataFrame(ex))
        
        # DEFINE VAR, SD, T SCORE
        # DIAGONAL OF COVARIANCE MATRIX SHOULD EQUAL AN ARRAY OF ALL COEF VAR
        var_b = newX.cov().to_numpy().diagonal().copy()
        var_b += .0000000001 #ADDING VERY SMALL NUMBER TO DEAL WITH 0s
        sd_b = np.sqrt(var_b)
        ts_b = params/ sd_b #T STATISTIC = (ESTIMATED COEF - HYPOTHESIZED COEF (0))/COEF SAMPLE STANDARD ERROR
        
        p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX) - 1))) for i in ts_b]
        p_values = np.round(p_values,3)
        
        genes_comp = {}

        for i in range(len(ex.columns)):
            gene = re.sub('gene_', '', re.sub('_non_neg_rawscore', '', ex.columns[i]))
            genes_comp[gene] = {}
            genes_comp[gene]['p_val'] = p_values[i + 1]
            
        random_best_p_values = []

        for i in range(10000):
            print(i)            
            temp_df = df_use.copy() #DON'T ALTER ORIG DF

            # GET CORRECTLY SIZED GROUPS, RANDOMLY ASSIGNED, SAME SPLITS AS IN ORIG
            ss_sized_group = temp_df.sample(n = len(set(df_gen[df_gen['outcome'] == 1]['sample_id'])))
            mf_sized_group = temp_df[~temp_df.index.isin(ss_sized_group.index.values)]

        #     RESET OUTCOME VALS BASED ON GROUP (NOW OUTCOME IS RANDOM)
            ss_sized_group['outcome'].values[:] = 1
            mf_sized_group['outcome'].values[:] = 0

        #     RECOMBINE INTO ONE DF
            new_df = pd.concat([ss_sized_group, mf_sized_group])
    
                # DEFINE X, Y, FITTED MODEL
            X = new_df.drop(columns = [x for x in new_df.columns if x == 'sample_id' or x == 'outcome'])
            y = new_df['outcome']
            
            model = model_type.fit(ex, why)
            
            params = np.append(model.intercept_, model.coef_)
            predictions = model.predict(ex)
            
            newX = pd.DataFrame({"Constant":np.ones(len(ex))}).join(pd.DataFrame(ex))
            
            var_b = newX.cov().to_numpy().diagonal().copy()
            var_b += .000000001
            sd_b = np.sqrt(var_b)
            ts_b = params/ sd_b
    
            p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX) - 1))) for i in ts_b]
            p_values = np.round(p_values,3)
            p_values = p_values[1:] #FIRST ONE WILL BE FOR CONSTANT (SHOULD EQUAL 0)
    
            baseline = np.min(p_values)
            random_best_p_values.append(baseline)
            
        for i in range(len(ex.columns)):
            gene = re.sub('gene_', '', re.sub('_non_neg_rawscore', '', ex.columns[i]))

            genes_comp[gene]['p_val'] = len([x for x in random_best_p_values if x < genes_comp[gene]['p_val']])/10000
    
        genes_comp_ordered = {k: v for k, v in sorted(genes_comp.items(), key=lambda item: item[1]['p_val'], reverse = False)}
        return genes_comp_ordered
    

            
    def tree_based(self, model_type, ex, why):
        
        model = model_type.fit(ex, why.values.ravel())
        
        genes_comp = {}

        for i in range(len(ex.columns)):
            gene = re.sub('gene_', '', re.sub('_non_neg_rawscore', '', ex.columns[i]))
            genes_comp[gene] = {}
            genes_comp[gene]['fi'] = model.feature_importances_[i]
            
        random_best_fi_vals = []

        for i in range(10000):
            print(i)            
            temp_df = df_use.copy() #DON'T ALTER ORIG DF

            ss_sized_group = temp_df.sample(n = len(set(df_gen[df_gen['outcome'] == 1]['sample_id'])))
            mf_sized_group = temp_df[~temp_df.index.isin(ss_sized_group.index.values)]

            ss_sized_group['outcome'].values[:] = 1
            mf_sized_group['outcome'].values[:] = 0

            new_df = pd.concat([ss_sized_group, mf_sized_group])

            X = new_df.drop(columns = [x for x in new_df.columns if x == 'sample_id' or x == 'outcome'])
            y = new_df['outcome']
            model = model_type.fit(ex, why)
            
            baseline = np.max(model.feature_importances_)
            random_best_fi_vals.append(baseline)
            
        for i in range(len(ex.columns)):
            gene = re.sub('gene_', '', re.sub('_non_neg_rawscore', '', ex.columns[i]))

            genes_comp[gene]['p_val'] = len([x for x in random_best_fi_vals if x > genes_comp[gene]['fi']])/10000
    
        genes_comp_filtered = {}

        for (key, value) in genes_comp.items():
           # Check if key is even then add pair to new dictionary
           if value['fi'] != 0:
               genes_comp_filtered[key] = value
            
        genes_comp_ordered = {k: v for k, v in sorted(genes_comp_filtered.items(), key=lambda item: item[1]['p_val'], reverse = False)}
        return genes_comp_ordered


In [24]:
print("LOGISTIC REGRESSION \n")
log_perm = perm_test()
log_perm_result = log_perm.linear(log, X, y)
print("RIDGE CLASSIFIER \n")
rdg_perm = perm_test()
rdg_perm_result = rdg_perm.linear(rdg, X, y)
print("ADABOOST \n")
ada_perm = perm_test()
ada_perm_result = ada_perm.tree_based(ada, X, y)
print("RANDOM FOREST \n")
rf_perm = perm_test()
rf_perm_result = rf_perm.tree_based(rf, X, y)

LOGISTIC REGRESSION 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271


1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056


3496
3497
3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695


5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334


6774
6775
6776
6777
6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973


...

2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217


3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857


5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390
5391
5392
5393
5394
5395
5396
5397
5398
5399
5400
5401
5402
5403
5404
5405
5406
5407
5408
5409
5410
5411
5412
5413
5414
5415
5416
5417
5418
5419
5420
5421
5422
5423
5424
5425
5426
5427
5428
5429
5430
5431
5432
5433
5434
5435
5436
5437
5438
5439
5440
5441
5442
5443
5444
5445
5446
5447
5448
5449
5450
5451
5452
5453
5454
5455
5456
5457
5458
5459
5460
5461
5462
5463
5464
5465
5466
5467
5468
5469
5470
5471
5472
5473
5474
5475
5476
5477
5478
5479
5480
5481
5482
5483
5484
5485
5486
5487
5488
5489
5490
5491
5492
5493
5494
5495
5496


6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022
7023
7024
7025
7026
7027
7028
7029
7030
7031
7032
7033
7034
7035
7036
7037
7038
7039
7040
7041
7042
7043
7044
7045
7046
7047
7048
7049
7050
7051
7052
7053
7054
7055
7056
7057
7058
7059
7060
7061
7062
7063
7064
7065
7066
7067
7068
7069
7070
7071
7072
7073
7074
7075
7076
7077
7078
7079
7080
7081
7082
7083
7084
7085
7086
7087
7088
7089
7090
7091
7092
7093
7094
7095
7096
7097
7098
7099
7100
7101
7102
7103
7104
7105
7106
7107
7108
7109
7110
7111
7112
7113
7114
7115
7116
7117
7118
7119
7120
7121
7122
7123
7124
7125
7126
7127
7128
7129
7130
7131
7132
7133
7134
7135
7136


8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622
8623
8624
8625
8626
8627
8628
8629
8630
8631
8632
8633
8634
8635
8636
8637
8638
8639
8640
8641
8642
8643
8644
8645
8646
8647
8648
8649
8650
8651
8652
8653
8654
8655
8656
8657
8658
8659
8660
8661
8662
8663
8664
8665
8666
8667
8668
8669
8670
8671
8672
8673
8674
8675
8676
8677
8678
8679
8680
8681
8682
8683
8684
8685
8686
8687
8688
8689
8690
8691
8692
8693
8694
8695
8696
8697
8698
8699
8700
8701
8702
8703
8704
8705
8706
8707
8708
8709
8710
8711
8712
8713
8714
8715
8716
8717
8718
8719
8720
8721
8722
8723
8724
8725
8726
8727
8728
8729
8730
8731
8732
8733
8734
8735
8736
8737
8738
8739
8740
8741
8742
8743
8744
8745
8746
8747
8748
8749
8750
8751
8752
8753
8754
8755
8756
8757
8758
8759
8760
8761
8762
8763
8764
8765
8766
8767
8768
8769
8770
8771
8772
8773
8774
8775


In [25]:
ada_df = pd.DataFrame.from_dict(ada_perm_result)
ada_df.to_csv("ada_dict.csv")
rf_df = pd.DataFrame.from_dict(rf_perm_result)
rf_df.to_csv("rf_dict.csv")
rdg_df = pd.DataFrame.from_dict(rdg_perm_result)
rdg_df.to_csv("rdg_dict.csv")
log_df = pd.DataFrame.from_dict(log_perm_result)
log_df.to_csv("log_dict.csv")

In [36]:
ada_perm_result
# ADABOOST - IQSEC1: FI = .1, P-VAL = .0
    

{'IQSEC1': {'fi': 0.1, 'p_val': 0.0},
 'ABCA6': {'fi': 0.02, 'p_val': 1.0},
 'ARMCX1': {'fi': 0.02, 'p_val': 1.0},
 'C1orf127': {'fi': 0.04, 'p_val': 1.0},
 'CDC42': {'fi': 0.04, 'p_val': 1.0},
 'CFHR2': {'fi': 0.04, 'p_val': 1.0},
 'CLUAP1': {'fi': 0.02, 'p_val': 1.0},
 'DDX60': {'fi': 0.02, 'p_val': 1.0},
 'FAT2': {'fi': 0.02, 'p_val': 1.0},
 'FAT3': {'fi': 0.06, 'p_val': 1.0},
 'IGFN1': {'fi': 0.02, 'p_val': 1.0},
 'IGSF10': {'fi': 0.02, 'p_val': 1.0},
 'KRTAP5-5': {'fi': 0.04, 'p_val': 1.0},
 'LAMA2': {'fi': 0.04, 'p_val': 1.0},
 'MARK1': {'fi': 0.02, 'p_val': 1.0},
 'NCOR2': {'fi': 0.02, 'p_val': 1.0},
 'NLGN3': {'fi': 0.04, 'p_val': 1.0},
 'NR3C1': {'fi': 0.02, 'p_val': 1.0},
 'OBSCN': {'fi': 0.02, 'p_val': 1.0},
 'ODF3L2': {'fi': 0.02, 'p_val': 1.0},
 'PCDH11X': {'fi': 0.04, 'p_val': 1.0},
 'PCDHGA8': {'fi': 0.02, 'p_val': 1.0},
 'PCLO': {'fi': 0.02, 'p_val': 1.0},
 'POSTN': {'fi': 0.02, 'p_val': 1.0},
 'SDK1': {'fi': 0.04, 'p_val': 1.0},
 'SETDB1': {'fi': 0.02, 'p_val': 1.0},
 

In [37]:
rf_perm_result
# RANDOM FOREST - NONE

{'KRTAP5-5': {'fi': 0.018182937145656025, 'p_val': 0.7615},
 'CDC42': {'fi': 0.016171985638887904, 'p_val': 0.9573},
 'ABCA6': {'fi': 0.014594474443221227, 'p_val': 0.9947},
 'ARMCX2': {'fi': 0.01287574042861618, 'p_val': 0.9998},
 'NCOR2': {'fi': 0.012906192647149772, 'p_val': 0.9998},
 'TPSD1': {'fi': 0.012695991087701409, 'p_val': 0.9999},
 '1-Mar': {'fi': 5.300620929867085e-07, 'p_val': 1.0},
 '2-Mar': {'fi': 0.001484773757753034, 'p_val': 1.0},
 'A2M': {'fi': 0.00039641972599870176, 'p_val': 1.0},
 'A2ML1': {'fi': 0.00031788472964943456, 'p_val': 1.0},
 'ABCA10': {'fi': 0.001421568627450981, 'p_val': 1.0},
 'ABCA13': {'fi': 3.715218784564135e-05, 'p_val': 1.0},
 'ABCA3': {'fi': 0.0009082006035721696, 'p_val': 1.0},
 'ABCA4': {'fi': 0.002535616133645992, 'p_val': 1.0},
 'ABCA8': {'fi': 2.800504314266973e-05, 'p_val': 1.0},
 'ABCA9': {'fi': 3.612865931706614e-06, 'p_val': 1.0},
 'ABCB5': {'fi': 0.00019944659238622, 'p_val': 1.0},
 'ABCB7': {'fi': 0.001873065015479877, 'p_val': 1.0},

In [38]:
log_perm_result
# LOGISTIC REGRESSION - KRTAP5-5: P-VAL = 0; TPSD1: P-VAL = 0

{'KRTAP5-5': {'p_val': 0.0},
 'TPSD1': {'p_val': 0.0},
 '1-Mar': {'p_val': 1.0},
 '1-Sep': {'p_val': 1.0},
 '11-Mar': {'p_val': 1.0},
 '2-Mar': {'p_val': 1.0},
 '4-Sep': {'p_val': 1.0},
 '6-Mar': {'p_val': 1.0},
 'A2M': {'p_val': 1.0},
 'A2ML1': {'p_val': 1.0},
 'A3GALT2': {'p_val': 1.0},
 'AADAC': {'p_val': 1.0},
 'AADACL4': {'p_val': 1.0},
 'AASDHPPT': {'p_val': 1.0},
 'AASS': {'p_val': 1.0},
 'AATK': {'p_val': 1.0},
 'ABCA10': {'p_val': 1.0},
 'ABCA13': {'p_val': 1.0},
 'ABCA3': {'p_val': 1.0},
 'ABCA4': {'p_val': 1.0},
 'ABCA5': {'p_val': 1.0},
 'ABCA6': {'p_val': 1.0},
 'ABCA7': {'p_val': 1.0},
 'ABCA8': {'p_val': 1.0},
 'ABCA9': {'p_val': 1.0},
 'ABCB1': {'p_val': 1.0},
 'ABCB4': {'p_val': 1.0},
 'ABCB5': {'p_val': 1.0},
 'ABCB7': {'p_val': 1.0},
 'ABCB9': {'p_val': 1.0},
 'ABCC1': {'p_val': 1.0},
 'ABCC10': {'p_val': 1.0},
 'ABCC11': {'p_val': 1.0},
 'ABCC12': {'p_val': 1.0},
 'ABCC3': {'p_val': 1.0},
 'ABCC4': {'p_val': 1.0},
 'ABCC6': {'p_val': 1.0},
 'ABCC8': {'p_val': 1.0},


In [39]:
rdg_perm_result
# RIDGE CLASSIFIER - KRTAP5-5: P-VAL = 0; TPSD1: P-VAL = 0

{'KRTAP5-5': {'p_val': 0.0},
 'TPSD1': {'p_val': 0.0},
 '1-Mar': {'p_val': 1.0},
 '1-Sep': {'p_val': 1.0},
 '11-Mar': {'p_val': 1.0},
 '2-Mar': {'p_val': 1.0},
 '4-Sep': {'p_val': 1.0},
 '6-Mar': {'p_val': 1.0},
 'A2M': {'p_val': 1.0},
 'A2ML1': {'p_val': 1.0},
 'A3GALT2': {'p_val': 1.0},
 'AADAC': {'p_val': 1.0},
 'AADACL4': {'p_val': 1.0},
 'AASDHPPT': {'p_val': 1.0},
 'AASS': {'p_val': 1.0},
 'AATK': {'p_val': 1.0},
 'ABCA10': {'p_val': 1.0},
 'ABCA13': {'p_val': 1.0},
 'ABCA3': {'p_val': 1.0},
 'ABCA4': {'p_val': 1.0},
 'ABCA5': {'p_val': 1.0},
 'ABCA6': {'p_val': 1.0},
 'ABCA7': {'p_val': 1.0},
 'ABCA8': {'p_val': 1.0},
 'ABCA9': {'p_val': 1.0},
 'ABCB1': {'p_val': 1.0},
 'ABCB4': {'p_val': 1.0},
 'ABCB5': {'p_val': 1.0},
 'ABCB7': {'p_val': 1.0},
 'ABCB9': {'p_val': 1.0},
 'ABCC1': {'p_val': 1.0},
 'ABCC10': {'p_val': 1.0},
 'ABCC11': {'p_val': 1.0},
 'ABCC12': {'p_val': 1.0},
 'ABCC3': {'p_val': 1.0},
 'ABCC4': {'p_val': 1.0},
 'ABCC6': {'p_val': 1.0},
 'ABCC8': {'p_val': 1.0},
